In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

In [ ]:
# Forcer la définition de la variable
os.environ['OPENAI_API_KEY'] = "sk-proj-"
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'  # Notez que j'ai corrigé gpt-4o en gpt-4

print("\nAprès définition directe :")
print("API Key:", os.getenv('OPENAI_API_KEY'))
print("Modèle:", os.getenv('OPENAI_MODEL_NAME'))

In [ ]:
import os

api_key = 'sk-proj-r'
print("API Key:", api_key)

model = os.getenv('OPENAI_MODEL_NAME')
print("Modèle configuré:", model)

In [4]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

In [5]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [12]:
from crewai.project import CrewBase, agent, crew, task

# Uncomment the following line to use an example of a custom tool
# from surprise_travel.tools.custom_tool import MyCustomTool

# Check our tools documentation for more information on how to use them
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from pydantic import BaseModel, Field
from typing import List, Optional

class Activity(BaseModel):
    name: str = Field(..., description="Name of the activity")
    location: str = Field(..., description="Location of the activity")
    description: str = Field(..., description="Description of the activity")
    date: str = Field(..., description="Date of the activity")
    cousine: str = Field(..., description="Cousine of the restaurant")
    why_its_suitable: str = Field(..., description="Why it's suitable for the traveler")
    reviews: Optional[List[str]] = Field(..., description="List of reviews")
    rating: Optional[float] = Field(..., description="Rating of the activity")

class DayPlan(BaseModel):
	date: str = Field(..., description="Date of the day")
	activities: List[Activity] = Field(..., description="List of activities")
	restaurants: List[str] = Field(..., description="List of restaurants")
	flight: Optional[str] = Field(None, description="Flight information")

class Itinerary(BaseModel):
  name: str = Field(..., description="Name of the itinerary, something funny")
  day_plans: List[DayPlan] = Field(..., description="List of day plans")
  hotel: str = Field(..., description="Hotel information")


In [ ]:
tool = ScrapeWebsiteTool(website_url='https://www.example.com')
toolrestau = ScrapeWebsiteTool(website_url='https://www.example.com')

In [13]:
# Creating Agents
personalized_activity_planner = Agent(
  config=agents_config['personalized_activity_planner'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()], # Example of custom tool, loaded at the beginning of file
  verbose=True,
  allow_delegation=False
)

restaurant_scout = Agent(
  config=agents_config['restaurant_scout'],
  tools=[SerperDevTool(), ScrapeWebsiteTool(website_url='https://www.lebey.com/les-adresses-du-12eme-arrondissement')],
  verbose=True,
  allow_delegation=False
)

itinerary_compiler = Agent(
  config=agents_config['itinerary_compiler'],
  tools=[SerperDevTool()],
  verbose=True,
  allow_delegation=False
)

In [14]:
# Creating Tasks
personalized_activity_planning_task = Task(
  config=tasks_config['personalized_activity_planning_task'],
  agent=personalized_activity_planner
)

restaurant_scenic_location_scout_task = Task(
  config=tasks_config['restaurant_scenic_location_scout_task'],
  agent=restaurant_scout
)

itinerary_compilation_task = Task(
  config=tasks_config['itinerary_compilation_task'],
  agent=itinerary_compiler,
  output_pydantic=Itinerary # This is the structured output we want
)

In [20]:
# Creating Crew
crew = Crew(
  agents=[
    personalized_activity_planner,
    restaurant_scout,
    itinerary_compiler
  ],
  tasks=[
    personalized_activity_planning_task,
    restaurant_scenic_location_scout_task,
    itinerary_compilation_task
  ],
  verbose=True
)

2024-12-20 10:44:41,224 - 15580 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [104]:
inputs = {
        'origin': 'São Paulo, GRU',
        'destination': 'New York, JFK',
        'age': 31,
        'hotel_location': 'Brooklyn',
        'flight_information': 'GOL 1234, leaving at June 30th, 2024, 10:00',
        'trip_duration': '14 days'
    }

In [16]:
inputs2 = {
        'origin': 'Saint Malo ,France',
        'destination': 'Paris , Charles de Gaule',
        'age': 50,
        'hotel_location': 'Paris 12 ème district',
        'flight_information': '  leaving at June 30th, 2024, 10:00',
        'trip_duration': '7 days'
    }

In [ ]:
# Run the crew
result = crew.kickoff(
  inputs=inputs
)

In [11]:
from IPython.display import display, Markdown

In [14]:
from IPython.display import display, Markdown

itinerary_md = """
#### **14-Day Itinerary to Brooklyn, New York**

---

#### **Day 1: Arrival in New York (JFK)**  
- **Flight Details:** Arrive at JFK International Airport.  
- **Hotel Check-in:** The Williamsburg Hotel  
  - Address: 96 Wythe Ave, Brooklyn, NY 11249  
  - Amenities: Rooftop pool, on-site restaurant, free Wi-Fi.
- **Evening Activity:** Walk along the East River waterfront.  
- **Dinner:** Visit **Peter Luger Steak House** for a classic Brooklyn dining experience.  

---

#### **Day 2: Exploring Williamsburg**  
- **Breakfast:** Have a meal at **Domino Park** café.  
- **Morning Activity:** Explore the Brooklyn Brewery and participate in a tasting tour.  
- **Lunch:** Enjoy a meal at **Smorgasburg** (open on weekends).  
- **Afternoon Activity:** Visit the **Brooklyn Flea Market**.  
- **Dinner:** Local cuisine at **Lilia** - famous for Italian pasta dishes.  

---

#### **Day 3: Brooklyn Heights and DUMBO**  
- **Breakfast:** **Clementine Bakery** for vegan options.  
- **Morning Activity:** Walk the Brooklyn Heights Promenade for stunning Manhattan views.  
- **Lunch:** **Time Out Market** for various food options.  
- **Afternoon Activity:** Visit **Jane’s Carousel** and explore the DUMBO area.  
- **Dinner:** **Atrium DUMBO** for seasonal American fare.  

---

#### **Day 4: Prospect Park and Brooklyn Museum**  
- **Breakfast:** **Baked in Brooklyn** for pastries and coffee.  
- **Morning Activity:** Visit **Prospect Park** and enjoy a leisurely stroll.  
- **Lunch:** **Prospect Park Picnic** - grab food from local vendors.  
- **Afternoon Activity:** Explore the **Brooklyn Museum**.  
- **Dinner:** Book a table at **The River Café** for a romantic view of the skyline.  

---

#### **Day 5: Cultural Exploration**  
- **Breakfast:** **Brewed Awakenings** coffee shop.  
- **Morning Activity:** Visit the **Brooklyn Botanic Garden**.  
- **Lunch:** **Frankies 457 Spuntino** famous for Italian sandwiches.  
- **Afternoon Activity:** Explore the **Brooklyn Academy of Music (BAM)**.  
- **Dinner:** Try **The Commodore** for Southern-inspired dishes.  

---

#### **Day 6: Relaxation Day**  
- **Breakfast:** Brunch at **Buttermilk Channel**.  
- **Morning Activity:** Spend the day at **Coney Island** (beach and boardwalk).  
- **Lunch:** Treat yourself to **Nathan's Famous** hotdogs.  
- **Afternoon Activity:** Ride the **Cyclone Roller Coaster**.  
- **Dinner:** Return to Brooklyn, dine at **The Clam**.  

---

#### **Day 7: Shopping and Exploring**  
- **Breakfast:** **Bagelsmith** for classic New York bagels.  
- **Morning Activity:** Shop along **Atlantic Avenue**.  
- **Lunch:** **Pasta e Vino** for a quick bite.  
- **Afternoon Activity:** Visit **Barclays Center** for events or the NBA store.  
- **Dinner:** **Emmy Squared** for award-winning pizza.  

---

#### **Day 8: Artistic Day**  
- **Breakfast:** **Sey Coffee** for artisanal coffee.  
- **Morning Activity:** Visit **Bushwick Street Art** for murals and a guided tour.  
- **Lunch:** **The Meatball Shop** for a casual meal.  
- **Afternoon Activity:** Explore local gallery exhibitions.  
- **Dinner:** **Roberta’s** for more pizza options.  

---

#### **Day 9: Day Trip to New York City**  
- **Breakfast:** Light breakfast at the hotel.  
- **Morning Activity:** Take the subway into Manhattan.  
- **Explore:** Central Park, Times Square, and 5th Avenue shopping.  
- **Lunch:** Eat at **Shake Shack** in Madison Square Park.  
- **Evening Activity:** See a show on **Broadway**.
- **Dinner:** Dinner at a Times Square restaurant of your choice.  

---

#### **Day 10: Brooklyn History**  
- **Breakfast:** Local bakery treats at **One Girl Cookies**.  
- **Morning Activity:** Visit the **Brooklyn Historical Society**.
- **Lunch:** Local flavors at **Junior’s** for famous cheesecake.
- **Afternoon Activity:** Stroll through **Green-Wood Cemetery**.  
- **Dinner:** **Bar LunÀtico** for live music and small plates.  

---

#### **Day 11: Foodie Tour**  
- **Breakfast:** **Café Regular** for coffee and pastries.  
- **Morning Activity:** Participate in a food tour through **Chinatown** and Brooklyn.  
- **Lunch:** Tour includes tastings from various local spots.  
- **Afternoon Activity:** Sweet treat at **Dough Doughnuts**.  
- **Dinner:** Dine at **Seafood City** for fresh seafood.  

---

#### **Day 12: Sports and Outdoor**  
- **Breakfast:** **Bakeri** for Scandinavian-style breakfast.  
- **Morning Activity:** Attend a game at **Barclays Center** or a local sporting event.  
- **Lunch:** **Friedman’s** for hearty, comfort food.  
- **Afternoon Activity:** Cycling around **Prospect Park**.  
- **Dinner:** **Frankies 457 Spuntino** for a cozy meal.  

---

#### **Day 13: Final Exploration**  
- **Breakfast:** Enjoy brunch at **Cafe Mogador**.  
- **Morning Activity:** Visit **Industry City** for shopping and art.  
- **Lunch:** Street food at **Smorgasburg** (if open).  
- **Afternoon Activity:** Relax at **Fort Greene Park**.  
- **Dinner:** Wrap up your trip at **L'Industrie Pizzeria** for a final taste of Brooklyn.  

---

#### **Day 14: Departure**  
- **Breakfast:** Last breakfast at the hotel or local café.  
- **Hotel Check-out:** The Williamsburg Hotel  
- **Flight Details:** Depart from JFK Airport.  

--- 
"""

display(Markdown(itinerary_md))


#### **14-Day Itinerary to Brooklyn, New York**

---

#### **Day 1: Arrival in New York (JFK)**  
- **Flight Details:** Arrive at JFK International Airport.  
- **Hotel Check-in:** The Williamsburg Hotel  
  - Address: 96 Wythe Ave, Brooklyn, NY 11249  
  - Amenities: Rooftop pool, on-site restaurant, free Wi-Fi.
- **Evening Activity:** Walk along the East River waterfront.  
- **Dinner:** Visit **Peter Luger Steak House** for a classic Brooklyn dining experience.  

---

#### **Day 2: Exploring Williamsburg**  
- **Breakfast:** Have a meal at **Domino Park** café.  
- **Morning Activity:** Explore the Brooklyn Brewery and participate in a tasting tour.  
- **Lunch:** Enjoy a meal at **Smorgasburg** (open on weekends).  
- **Afternoon Activity:** Visit the **Brooklyn Flea Market**.  
- **Dinner:** Local cuisine at **Lilia** - famous for Italian pasta dishes.  

---

#### **Day 3: Brooklyn Heights and DUMBO**  
- **Breakfast:** **Clementine Bakery** for vegan options.  
- **Morning Activity:** Walk the Brooklyn Heights Promenade for stunning Manhattan views.  
- **Lunch:** **Time Out Market** for various food options.  
- **Afternoon Activity:** Visit **Jane’s Carousel** and explore the DUMBO area.  
- **Dinner:** **Atrium DUMBO** for seasonal American fare.  

---

#### **Day 4: Prospect Park and Brooklyn Museum**  
- **Breakfast:** **Baked in Brooklyn** for pastries and coffee.  
- **Morning Activity:** Visit **Prospect Park** and enjoy a leisurely stroll.  
- **Lunch:** **Prospect Park Picnic** - grab food from local vendors.  
- **Afternoon Activity:** Explore the **Brooklyn Museum**.  
- **Dinner:** Book a table at **The River Café** for a romantic view of the skyline.  

---

#### **Day 5: Cultural Exploration**  
- **Breakfast:** **Brewed Awakenings** coffee shop.  
- **Morning Activity:** Visit the **Brooklyn Botanic Garden**.  
- **Lunch:** **Frankies 457 Spuntino** famous for Italian sandwiches.  
- **Afternoon Activity:** Explore the **Brooklyn Academy of Music (BAM)**.  
- **Dinner:** Try **The Commodore** for Southern-inspired dishes.  

---

#### **Day 6: Relaxation Day**  
- **Breakfast:** Brunch at **Buttermilk Channel**.  
- **Morning Activity:** Spend the day at **Coney Island** (beach and boardwalk).  
- **Lunch:** Treat yourself to **Nathan's Famous** hotdogs.  
- **Afternoon Activity:** Ride the **Cyclone Roller Coaster**.  
- **Dinner:** Return to Brooklyn, dine at **The Clam**.  

---

#### **Day 7: Shopping and Exploring**  
- **Breakfast:** **Bagelsmith** for classic New York bagels.  
- **Morning Activity:** Shop along **Atlantic Avenue**.  
- **Lunch:** **Pasta e Vino** for a quick bite.  
- **Afternoon Activity:** Visit **Barclays Center** for events or the NBA store.  
- **Dinner:** **Emmy Squared** for award-winning pizza.  

---

#### **Day 8: Artistic Day**  
- **Breakfast:** **Sey Coffee** for artisanal coffee.  
- **Morning Activity:** Visit **Bushwick Street Art** for murals and a guided tour.  
- **Lunch:** **The Meatball Shop** for a casual meal.  
- **Afternoon Activity:** Explore local gallery exhibitions.  
- **Dinner:** **Roberta’s** for more pizza options.  

---

#### **Day 9: Day Trip to New York City**  
- **Breakfast:** Light breakfast at the hotel.  
- **Morning Activity:** Take the subway into Manhattan.  
- **Explore:** Central Park, Times Square, and 5th Avenue shopping.  
- **Lunch:** Eat at **Shake Shack** in Madison Square Park.  
- **Evening Activity:** See a show on **Broadway**.
- **Dinner:** Dinner at a Times Square restaurant of your choice.  

---

#### **Day 10: Brooklyn History**  
- **Breakfast:** Local bakery treats at **One Girl Cookies**.  
- **Morning Activity:** Visit the **Brooklyn Historical Society**.
- **Lunch:** Local flavors at **Junior’s** for famous cheesecake.
- **Afternoon Activity:** Stroll through **Green-Wood Cemetery**.  
- **Dinner:** **Bar LunÀtico** for live music and small plates.  

---

#### **Day 11: Foodie Tour**  
- **Breakfast:** **Café Regular** for coffee and pastries.  
- **Morning Activity:** Participate in a food tour through **Chinatown** and Brooklyn.  
- **Lunch:** Tour includes tastings from various local spots.  
- **Afternoon Activity:** Sweet treat at **Dough Doughnuts**.  
- **Dinner:** Dine at **Seafood City** for fresh seafood.  

---

#### **Day 12: Sports and Outdoor**  
- **Breakfast:** **Bakeri** for Scandinavian-style breakfast.  
- **Morning Activity:** Attend a game at **Barclays Center** or a local sporting event.  
- **Lunch:** **Friedman’s** for hearty, comfort food.  
- **Afternoon Activity:** Cycling around **Prospect Park**.  
- **Dinner:** **Frankies 457 Spuntino** for a cozy meal.  

---

#### **Day 13: Final Exploration**  
- **Breakfast:** Enjoy brunch at **Cafe Mogador**.  
- **Morning Activity:** Visit **Industry City** for shopping and art.  
- **Lunch:** Street food at **Smorgasburg** (if open).  
- **Afternoon Activity:** Relax at **Fort Greene Park**.  
- **Dinner:** Wrap up your trip at **L'Industrie Pizzeria** for a final taste of Brooklyn.  

---

#### **Day 14: Departure**  
- **Breakfast:** Last breakfast at the hotel or local café.  
- **Hotel Check-out:** The Williamsburg Hotel  
- **Flight Details:** Depart from JFK Airport.  

--- 


In [12]:
# Run the crew
result = crew.kickoff(
  inputs=inputs2
)

# Agent: Activity Planner
## Task: Research and find cool things to do at Paris , Charles de Gaule. Focus on activities and events that match the traveler's interests and age group. Utilize internet search tools and recommendation engines to gather the information.

Traveler's information:

- origin: Saint Malo ,France
- destination: Paris , Charles de Gaule
- age of the traveler: 50
- hotel localtion: Paris 12 ème district
- flight infromation:   leaving at June 30th, 2024, 10:00
- how long is the trip: 7 days



# Agent: Activity Planner
## Thought: To create a tailored itinerary for the traveler, I need to search for events and activities around Paris, specifically in the 12th district and nearby areas. I'll start by searching for popular activities and events happening in Paris during the travel dates.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"things to do in Paris 12th district June 2024\"}"
## Tool Output: 
{'message': 'Unauthorized.', 'statusCode':

In [16]:
itinerary_md1 = """
**Day 1: Arrival and Evening Stroll**
- **Arrival at Charles de Gaulle Airport**
- **Hotel Check-in**: [Specify Hotel Information]
- **Activity:** Explore the *Marais District*
  - **Location:** Marais District, 75003 Paris, France
  - **Description:** Discover the charming streets of the Marais, known for its boutiques, art galleries, and cafés. A relaxed introduction to Paris, perfect for unwinding after your journey.
  
- **Dinner:** **Les Philosophes**  
  - **Rating:** 4.5/5 (TripAdvisor)  
  - **Cuisine:** Traditional French  
  - **Address:** 28 Rue Vieille du Temple, 75004 Paris, France  
  - **Description:** A charming bistro serving classic French dishes in a casual setting.

---

**Day 2: Historical Sights**
- **Activity:** Visit the *Louvre Museum*
  - **Location:** Rue de Rivoli, 75001 Paris, France
  - **Description:** Home to thousands of works of art including the *Mona Lisa* and *Venus de Milo*. Ideal for art lovers; a guided tour can provide insights into the collections.

- **Activity:** *Seine River Cruise*
  - **Location:** Seine River
  - **Description:** An evening cruise with dinner, offering stunning views of iconic landmarks at night.

---

**Day 3: Culinary Day**
- **Activity:** *Cooking Class*
  - **Location:** Le Foodist, near the Seine
  - **Description:** Hands-on French cooking class focusing on classic recipes. Great for food enthusiasts who enjoy culinary experiences.

- **Dinner:** **Le Train Bleu**  
  - **Rating:** 4.6/5 (TripAdvisor)  
  - **Cuisine:** French  
  - **Address:** Place Louis Armand, 75012 Paris, France  
  - **Description:** Celebrated for its ornate Belle Époque decor and exquisite traditional French cuisine.

---

**Day 4: Cultural Experiences**
- **Activity:** Visit the *Musée d'Orsay*
  - **Location:** Rue de la Légion d'Honneur, 75007 Paris, France
  - **Description:** Discover noteworthy artworks from the Impressionist era, perfect for those with an appreciation for art history.

- **Evening Activity:** Attend a performance at the *Opéra Bastille* (Check for current schedule).

---

**Day 5: Garden Day**
- **Activity:** *Stroll through Jardin du Luxembourg*
  - **Location:** Boulevard Saint-Michel, 75006 Paris, France
  - **Description:** Beautiful gardens, fountains, and the picturesque Senate building, perfect for relaxing and enjoying nature.

- **Optional Activity:** Visit the *Pantheon* to delve into French history and notable figures.

---

**Day 6: Day Trip Option**
- **Activity:** *Versailles Palace*
  - **Location:** Versailles
  - **Description:** Explore the grandeur of the palace and its expansive gardens. Ideal for history buffs.

- **Evening Option:** Dinner at a restaurant near the hotel, exploring local cuisine.

---

**Day 7: Local Favorites and Departure**
- **Activity:** Explore the *Bastille Market*
  - **Location:** Place de la Bastille (open Thursdays and Sundays)
  - **Description:** Delight in fresh produce and unique food items at this vibrant market, allowing for interaction with locals and indulging in the Parisian lifestyle.

- **Final Lunch:** **Le Baron Rouge**  
  - **Rating:** 4.5/5 (TripAdvisor)  
  - **Cuisine:** French, Wine Bar  
  - **Address:** 1 Rue Théophile Roussel, 75012 Paris, France  
  - **Description:** A classic local wine bar offering small plates to enjoy before departure.

---

**Notes:**
- Please fill in specific flight details and hotel accommodations.
- Consider making reservations for dining experiences and tours in advance.
- Check opening hours and availability of activities in advance.

This itinerary blends culinary experiences, cultural exploration, and scenic beauty, tailored for a traveler seeking both relaxation and enrichment in Paris. Enjoy your trip!  

--- 
"""

In [14]:
itinerary_md2 = """
**Day 1: Arrival and Exploration of Paris 12th District**
- **Activity:** Visit Parc de Bercy
  - **Location:** Paris 12th District
  - **Description:** A beautiful and peaceful park divided into three different gardens with modern touches and art installations.
  - **Why it's suitable:** Perfect for a relaxing stroll after a journey, offering calm green spaces ideal for a traveler's first day in Paris.
  - **Reviews and Ratings:** Generally rated 4.5/5 for its serenity and picturesque environment.

- **Activity:** Eat at Bercy Village
  - **Location:** Paris 12th District
  - **Description:** A quaint shopping street with boutiques and cafes located next to Parc de Bercy.
  - **Why it's suitable:** Offers a local dining experience with various culinary options which would be comfortable for a relaxing evening.
  - **Reviews and Ratings:** Visitors appreciate the ambiance and rate it around 4/5.

**Day 2: Cultural Immersion**
- **Activity:** Visit the Palais de la Porte Dorée
  - **Location:** 293 Avenue Daumesnil, Paris 12th District
  - **Description:** A historical monument with an art deco style, hosting the Musée de l'Histoire de l'Immigration and an Aquarium.
  - **Why it's suitable:** Offers educational and cultural insights, suitable for adults interested in history and architecture.
  - **Reviews and Ratings:** Rated 4.5/5 for its rich exhibits and impressive architectural design.

**Day 3: Gastronomy and Shopping**
- **Activity:** Le Marché d'Aligre
  - **Location:** Place d'Aligre, Paris 12th District
  - **Description:** A lively market with fresh produce, cheese, wine, and a flea market.
  - **Why it's suitable:** Offers an authentic Parisian shopping experience with opportunities to taste local delicacies.
  - **Reviews and Ratings:** Popular for its vibrant atmosphere, with a rating of 4.6/5.

**Day 4: A Trip Along the Seine**
- **Activity:** Seine River Cruise
  - **Location:** Departs from multiple points, nearest from Port de la Rapée, Paris 12th
  - **Description:** A boat tour offering views of iconic landmarks like Notre-Dame Cathedral and the Eiffel Tower.
  - **Why it's suitable:** Provides a relaxing and scenic view of the city, fitting for a comfortable outing.
  - **Reviews and Ratings:** Generally well-received with a 4.7/5 average rating, praised for scenic beauty.

**Day 5: Art and Creativity**
- **Activity:** The Viaduc des Arts
  - **Location:** Paris 12th District along the Coulée verte René-Dumont
  - **Description:** A railway viaduct converted into arts and crafts shops.
  - **Why it's suitable:** Perfect for appreciating local artistry and finding unique gifts.
  - **Reviews and Ratings:** Visitors appreciate the creativity, rating it 4.4/5.

**Day 6: Day Trip to Nearby Château de Vincennes**
- **Activity:** Explore Château de Vincennes
  - **Location:** Avenue de Paris, 94300 Vincennes, near Paris
  - **Description:** A historical royal fortress with a medieval keep and expansive greenery.
  - **Why it's suitable:** Offers a historical exploration opportunity with beautiful gardens.
  - **Reviews and Ratings:** Rated 4.5/5 for its historical significance and well-preserved structures.

**Day 7: Leisure and Departure Preparation**
- **Activity:** Relax at Bois de Vincennes
  - **Location:** Paris, near the 12th District
  - **Description:** The largest public park in the city with beautiful lakes and the Parc Floral.
  - **Why it's suitable:** A perfect place for a leisurely day in nature to unwind before departure.
  - **Reviews and Ratings:** Known for its expansive green areas and tranquility, rated 4.6/5.

This itinerary caters to the traveler's interest in cultural, historical, and leisurely activities that are age-appropriate while balancing relaxation and exploration.-- 
"""

In [15]:
from IPython.display import display, Markdown
display(Markdown(itinerary_md2))


**Day 1: Arrival and Exploration of Paris 12th District**
- **Activity:** Visit Parc de Bercy
  - **Location:** Paris 12th District
  - **Description:** A beautiful and peaceful park divided into three different gardens with modern touches and art installations.
  - **Why it's suitable:** Perfect for a relaxing stroll after a journey, offering calm green spaces ideal for a traveler's first day in Paris.
  - **Reviews and Ratings:** Generally rated 4.5/5 for its serenity and picturesque environment.

- **Activity:** Eat at Bercy Village
  - **Location:** Paris 12th District
  - **Description:** A quaint shopping street with boutiques and cafes located next to Parc de Bercy.
  - **Why it's suitable:** Offers a local dining experience with various culinary options which would be comfortable for a relaxing evening.
  - **Reviews and Ratings:** Visitors appreciate the ambiance and rate it around 4/5.

**Day 2: Cultural Immersion**
- **Activity:** Visit the Palais de la Porte Dorée
  - **Location:** 293 Avenue Daumesnil, Paris 12th District
  - **Description:** A historical monument with an art deco style, hosting the Musée de l'Histoire de l'Immigration and an Aquarium.
  - **Why it's suitable:** Offers educational and cultural insights, suitable for adults interested in history and architecture.
  - **Reviews and Ratings:** Rated 4.5/5 for its rich exhibits and impressive architectural design.

**Day 3: Gastronomy and Shopping**
- **Activity:** Le Marché d'Aligre
  - **Location:** Place d'Aligre, Paris 12th District
  - **Description:** A lively market with fresh produce, cheese, wine, and a flea market.
  - **Why it's suitable:** Offers an authentic Parisian shopping experience with opportunities to taste local delicacies.
  - **Reviews and Ratings:** Popular for its vibrant atmosphere, with a rating of 4.6/5.

**Day 4: A Trip Along the Seine**
- **Activity:** Seine River Cruise
  - **Location:** Departs from multiple points, nearest from Port de la Rapée, Paris 12th
  - **Description:** A boat tour offering views of iconic landmarks like Notre-Dame Cathedral and the Eiffel Tower.
  - **Why it's suitable:** Provides a relaxing and scenic view of the city, fitting for a comfortable outing.
  - **Reviews and Ratings:** Generally well-received with a 4.7/5 average rating, praised for scenic beauty.

**Day 5: Art and Creativity**
- **Activity:** The Viaduc des Arts
  - **Location:** Paris 12th District along the Coulée verte René-Dumont
  - **Description:** A railway viaduct converted into arts and crafts shops.
  - **Why it's suitable:** Perfect for appreciating local artistry and finding unique gifts.
  - **Reviews and Ratings:** Visitors appreciate the creativity, rating it 4.4/5.

**Day 6: Day Trip to Nearby Château de Vincennes**
- **Activity:** Explore Château de Vincennes
  - **Location:** Avenue de Paris, 94300 Vincennes, near Paris
  - **Description:** A historical royal fortress with a medieval keep and expansive greenery.
  - **Why it's suitable:** Offers a historical exploration opportunity with beautiful gardens.
  - **Reviews and Ratings:** Rated 4.5/5 for its historical significance and well-preserved structures.

**Day 7: Leisure and Departure Preparation**
- **Activity:** Relax at Bois de Vincennes
  - **Location:** Paris, near the 12th District
  - **Description:** The largest public park in the city with beautiful lakes and the Parc Floral.
  - **Why it's suitable:** A perfect place for a leisurely day in nature to unwind before departure.
  - **Reviews and Ratings:** Known for its expansive green areas and tranquility, rated 4.6/5.

This itinerary caters to the traveler's interest in cultural, historical, and leisurely activities that are age-appropriate while balancing relaxation and exploration.-- 


In [17]:
from IPython.display import display, Markdown
display(Markdown(itinerary_md1))


**Day 1: Arrival and Evening Stroll**
- **Arrival at Charles de Gaulle Airport**
- **Hotel Check-in**: [Specify Hotel Information]
- **Activity:** Explore the *Marais District*
  - **Location:** Marais District, 75003 Paris, France
  - **Description:** Discover the charming streets of the Marais, known for its boutiques, art galleries, and cafés. A relaxed introduction to Paris, perfect for unwinding after your journey.
  
- **Dinner:** **Les Philosophes**  
  - **Rating:** 4.5/5 (TripAdvisor)  
  - **Cuisine:** Traditional French  
  - **Address:** 28 Rue Vieille du Temple, 75004 Paris, France  
  - **Description:** A charming bistro serving classic French dishes in a casual setting.

---

**Day 2: Historical Sights**
- **Activity:** Visit the *Louvre Museum*
  - **Location:** Rue de Rivoli, 75001 Paris, France
  - **Description:** Home to thousands of works of art including the *Mona Lisa* and *Venus de Milo*. Ideal for art lovers; a guided tour can provide insights into the collections.

- **Activity:** *Seine River Cruise*
  - **Location:** Seine River
  - **Description:** An evening cruise with dinner, offering stunning views of iconic landmarks at night.

---

**Day 3: Culinary Day**
- **Activity:** *Cooking Class*
  - **Location:** Le Foodist, near the Seine
  - **Description:** Hands-on French cooking class focusing on classic recipes. Great for food enthusiasts who enjoy culinary experiences.

- **Dinner:** **Le Train Bleu**  
  - **Rating:** 4.6/5 (TripAdvisor)  
  - **Cuisine:** French  
  - **Address:** Place Louis Armand, 75012 Paris, France  
  - **Description:** Celebrated for its ornate Belle Époque decor and exquisite traditional French cuisine.

---

**Day 4: Cultural Experiences**
- **Activity:** Visit the *Musée d'Orsay*
  - **Location:** Rue de la Légion d'Honneur, 75007 Paris, France
  - **Description:** Discover noteworthy artworks from the Impressionist era, perfect for those with an appreciation for art history.

- **Evening Activity:** Attend a performance at the *Opéra Bastille* (Check for current schedule).

---

**Day 5: Garden Day**
- **Activity:** *Stroll through Jardin du Luxembourg*
  - **Location:** Boulevard Saint-Michel, 75006 Paris, France
  - **Description:** Beautiful gardens, fountains, and the picturesque Senate building, perfect for relaxing and enjoying nature.

- **Optional Activity:** Visit the *Pantheon* to delve into French history and notable figures.

---

**Day 6: Day Trip Option**
- **Activity:** *Versailles Palace*
  - **Location:** Versailles
  - **Description:** Explore the grandeur of the palace and its expansive gardens. Ideal for history buffs.

- **Evening Option:** Dinner at a restaurant near the hotel, exploring local cuisine.

---

**Day 7: Local Favorites and Departure**
- **Activity:** Explore the *Bastille Market*
  - **Location:** Place de la Bastille (open Thursdays and Sundays)
  - **Description:** Delight in fresh produce and unique food items at this vibrant market, allowing for interaction with locals and indulging in the Parisian lifestyle.

- **Final Lunch:** **Le Baron Rouge**  
  - **Rating:** 4.5/5 (TripAdvisor)  
  - **Cuisine:** French, Wine Bar  
  - **Address:** 1 Rue Théophile Roussel, 75012 Paris, France  
  - **Description:** A classic local wine bar offering small plates to enjoy before departure.

---

**Notes:**
- Please fill in specific flight details and hotel accommodations.
- Consider making reservations for dining experiences and tours in advance.
- Check opening hours and availability of activities in advance.

This itinerary blends culinary experiences, cultural exploration, and scenic beauty, tailored for a traveler seeking both relaxation and enrichment in Paris. Enjoy your trip!  

--- 


In [21]:
inputs3 = {
        'origin': 'Saint Malo ,France',
        'destination': 'Paris , Charles de Gaule',
        'age': 50,
        'hotel_location': 'Paris 12 ème district',
        'flight_information': '  leaving at June 30th, 2024, 10:00',
        'trip_duration': '7 days'
    }

In [22]:
# Run the crew
result = crew.kickoff(
  inputs=inputs3
)

# Agent: Activity Planner
## Task: Research and find cool things to do at Paris , Charles de Gaule. Focus on activities and events that match the traveler's interests and age group. Utilize internet search tools and recommendation engines to gather the information.

Traveler's information:

- origin: Saint Malo ,France
- destination: Paris , Charles de Gaule
- age of the traveler: 50
- hotel localtion: Paris 12 ème district
- flight infromation:   leaving at June 30th, 2024, 10:00
- how long is the trip: 7 days



# Agent: Activity Planner
## Thought: I need to gather information about activities and events in Paris that would be suitable for a 50-year-old traveler staying in the 12th district for 7 days.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"things to do in Paris for 50 year old traveler June 2024\"}"
## Tool Output: 
{'message': 'Unauthorized.', 'statusCode': 403}


# Agent: Activity Planner
## Using tool: Search the internet
## Tool Input: 
"{\"sea

In [26]:
from IPython.display import display, Markdown

itinerary_md = """
## **7-Day Paris Itinerary**

### **Day 1: Arrival in Paris**
- **Flight**: Recommended flights include Air France and Delta, with direct options from major cities like New York to Charles de Gaulle Airport.
- **Hotel**: Check-in at **Hotel Le Meurice** (228 Rue de Rivoli, 75001 Paris). A luxury hotel located near many attractions.
- **Activity**: 
  - **Seine River Night Cruise**
    - **Location**: Port de la Bourdonnais
    - **Description**: Relax and enjoy beautiful views of Paris's landmarks illuminated at night.
- **Dining**: Enjoy dinner at a nearby bistro such as **Le Relais de l'Entrecôte**.

---

### **Day 2: Exploration of the 12th District**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Visit Parc de Bercy**
    - **Location**: 8 Cour de Bercy
    - **Description**: Enjoy a leisurely stroll or have a picnic in the park's beautiful gardens.
- **Dining**: 
  - **Lunch**: **Bistrot Jouvence**
    - **Address**: 172 bis, rue du Faubourg Saint-Antoine, 75012 Paris
- **Evening**: Stroll through local shops and markets.

---

### **Day 3: Culture and History**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Visit the Palais Omnisports de Paris-Bercy**
    - **Location**: 8 Boulevard de Bercy
    - **Description**: Check for events like concerts during your visit.
- **Dining**: 
  - **Dinner**: **Restaurant Passerini**
    - **Address**: 65, rue Traversière, 75012 Paris

---

### **Day 4: Art and Museums**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Explore the Musée des Arts Forains**
    - **Location**: 53 Avenue des Terroirs de France
    - **Description**: An engaging and interactive carnival arts museum.
- **Dining**: 
  - **Dinner**: **Bistro S**
    - **Address**: 7, rue Saint-Nicolas, 75012 Paris

---

### **Day 5: Culinary Delights**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Culinary Class or Food Tour in the 12th District**
    - **Description**: Participate in hands-on cooking classes or guided food tours.
- **Dining**: Enjoy local delicacies during the tour.

---

### **Day 6: Day Trip to Nearby Attractions**
- **Hotel**: Continue your stay at Hotel Le Meurice (or consider Hôtel le Belleval for a change).
- **Activity**: 
  - **Visit Château de Vincennes**
    - **Location**: 2 Avenue de Paris, Vincennes
    - **Description**: A historic castle with beautiful grounds.
- **Dining**: 
  - **Dinner**: **Au Trou Gascon**
    - **Address**: 40, rue Taine, 75012 Paris

---

### **Day 7: Leisure and Wrap-up**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Stroll through Marché d'Aligre**
    - **Location**: Place d'Aligre
    - **Description**: A vibrant market for last-minute shopping.
- **Dining**: 
  - **Lunch**: **Bistro Godaille**
    - **Address**: 9, rue Antoine Vollon, 75012 Paris

Enjoy your trip and indulge in the culinary and cultural treasures that Paris has to offer!
"""

display(Markdown(itinerary_md))


## **7-Day Paris Itinerary**

### **Day 1: Arrival in Paris**
- **Flight**: Recommended flights include Air France and Delta, with direct options from major cities like New York to Charles de Gaulle Airport.
- **Hotel**: Check-in at **Hotel Le Meurice** (228 Rue de Rivoli, 75001 Paris). A luxury hotel located near many attractions.
- **Activity**: 
  - **Seine River Night Cruise**
    - **Location**: Port de la Bourdonnais
    - **Description**: Relax and enjoy beautiful views of Paris's landmarks illuminated at night.
- **Dining**: Enjoy dinner at a nearby bistro such as **Le Relais de l'Entrecôte**.

---

### **Day 2: Exploration of the 12th District**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Visit Parc de Bercy**
    - **Location**: 8 Cour de Bercy
    - **Description**: Enjoy a leisurely stroll or have a picnic in the park's beautiful gardens.
- **Dining**: 
  - **Lunch**: **Bistrot Jouvence**
    - **Address**: 172 bis, rue du Faubourg Saint-Antoine, 75012 Paris
- **Evening**: Stroll through local shops and markets.

---

### **Day 3: Culture and History**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Visit the Palais Omnisports de Paris-Bercy**
    - **Location**: 8 Boulevard de Bercy
    - **Description**: Check for events like concerts during your visit.
- **Dining**: 
  - **Dinner**: **Restaurant Passerini**
    - **Address**: 65, rue Traversière, 75012 Paris

---

### **Day 4: Art and Museums**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Explore the Musée des Arts Forains**
    - **Location**: 53 Avenue des Terroirs de France
    - **Description**: An engaging and interactive carnival arts museum.
- **Dining**: 
  - **Dinner**: **Bistro S**
    - **Address**: 7, rue Saint-Nicolas, 75012 Paris

---

### **Day 5: Culinary Delights**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Culinary Class or Food Tour in the 12th District**
    - **Description**: Participate in hands-on cooking classes or guided food tours.
- **Dining**: Enjoy local delicacies during the tour.

---

### **Day 6: Day Trip to Nearby Attractions**
- **Hotel**: Continue your stay at Hotel Le Meurice (or consider Hôtel le Belleval for a change).
- **Activity**: 
  - **Visit Château de Vincennes**
    - **Location**: 2 Avenue de Paris, Vincennes
    - **Description**: A historic castle with beautiful grounds.
- **Dining**: 
  - **Dinner**: **Au Trou Gascon**
    - **Address**: 40, rue Taine, 75012 Paris

---

### **Day 7: Leisure and Wrap-up**
- **Hotel**: Continue your stay at Hotel Le Meurice.
- **Activity**: 
  - **Stroll through Marché d'Aligre**
    - **Location**: Place d'Aligre
    - **Description**: A vibrant market for last-minute shopping.
- **Dining**: 
  - **Lunch**: **Bistro Godaille**
    - **Address**: 9, rue Antoine Vollon, 75012 Paris

Enjoy your trip and indulge in the culinary and cultural treasures that Paris has to offer!


In [27]:
from IPython.display import display, Markdown

itinerary_md = """
### Day 1: Arrival and Night Cruise
- **Activity Name**: Seine River Night Cruise
- **Location**: Port de la Bourdonnais
- **Description**: Relax and enjoy a beautiful view of Paris's landmarks illuminated at night on a comfortable boat ride along the Seine River.
- **Why Suitable**: Perfect to unwind after traveling, ideal for adults who appreciate scenic experiences.
- **Potential Reviews**: Highly rated for romantic ambiance and unique perspectives of the Eiffel Tower and Notre Dame.

### Day 2: Exploration of the 12th District
- **Activity Name**: Visit Parc de Bercy 
- **Location**: 8 Cour de Bercy
- **Description**: Enjoy a morning stroll or a picnic in the park featuring beautiful gardens, open spaces, and outdoor activities.
- **Why Suitable**: Offers a peaceful atmosphere, great for relaxation and leisurely walks.
- **Potential Reviews**: Visitors praise its tranquility and lovely gardens.

- **Restaurant Recommendation**: **Bistrot Jouvence**  
  **Address**: 172 bis, rue du Faubourg Saint-Antoine, 75012 Paris  
  **Cuisine Type**: French Bistro  
  **Description**: One of the best bistros in the city, set in a beautifully preserved former pharmacy.  
  **Rating**: Highly rated for atmosphere and traditional cuisine.

### Day 3: Culture and History
- **Activity Name**: Visit the Palais Omnisports de Paris-Bercy 
- **Location**: 8 Boulevard de Bercy
- **Description**: Check out the event schedule for concerts or shows occurring during the trip.
- **Why Suitable**: Great for those who enjoy live entertainment.
- **Potential Reviews**: Known for excellent acoustics and a variety of events.

- **Restaurant Recommendation**: **Restaurant Passerini**  
  **Address**: 65, rue Traversière, 75012 Paris  
  **Cuisine Type**: Italian  
  **Description**: Known for a cozy atmosphere with exquisite Italian cuisine, very popular among locals.  
  **Rating**: Highly regarded for authentic flavors and quality.

### Day 4: Art and Museums
- **Activity Name**: Explore the Musée des Arts Forains
- **Location**: 53 Avenue des Terroirs de France
- **Description**: An interactive museum showcasing carnival arts and fairground attractions.
- **Why Suitable**: Engaging and nostalgic, perfect for culture lovers.
- **Potential Reviews**: Visitors find it enchanting and appreciate the hands-on nature of the exhibits.

- **Restaurant Recommendation**: **Bistro S**  
  **Address**: 7, rue Saint-Nicolas, 75012 Paris  
  **Cuisine Type**: Japanese-Fusion  
  **Description**: A unique bistro offering a blend of Japanese and French cuisines.  
  **Rating**: Highly praised for creativity and taste.

### Day 5: Culinary Delights
- **Activity Name**: Culinary Class or Food Tour in the 12th District  
- **Location**: Various cooking schools or culinary spots  
- **Description**: Participate in a French cooking class or take a guided food tour sampling local treats.
- **Why Suitable**: Ideal for food lovers wanting to indulge.
- **Potential Reviews**: Praised for knowledgeable instructors and delicious outcomes.

### Day 6: Day Trip to Nearby Attractions
- **Activity Name**: Visit Château de Vincennes
- **Location**: 2 Avenue de Paris, Vincennes
- **Description**: Explore the historic castle with beautiful grounds and architectural significance.
- **Why Suitable**: A blend of history and peaceful surroundings.
- **Potential Reviews**: Noted for impressive architecture and less crowded nature.

- **Restaurant Recommendation**: **Au Trou Gascon**  
  **Address**: 40, rue Taine, 75012 Paris  
  **Cuisine Type**: French  
  **Description**: A well-known spot for hearty French cooking, famous for its traditional recipes.  
  **Rating**: Very well-reviewed for quality and ambiance.

### Day 7: Leisure and Wrap-up
- **Activity Name**: Stroll through Marché d'Aligre 
- **Location**: Place d'Aligre
- **Description**: A vibrant market perfect for last-minute exploration and cultural immersion.
- **Why Suitable**: Engaging atmosphere, ideal for picking up souvenirs and fresh produce.
- **Potential Reviews**: Loved for its local vibe and variety of offerings.

- **Restaurant Recommendation**: **Bistro Godaille**  
  **Address**: 9, rue Antoine Vollon, 75012 Paris  
  **Cuisine Type**: French Bistro  
  **Description**: A cozy bistro known for its casual vibe and delicious meals, conveniently located.  
  **Rating**: Highly praised by regulars for its welcoming atmosphere and tasty dishes.

Enjoy the trip and indulge in the culinary and cultural treasures that Paris, especially the 12th district, has to offer!
"""

display(Markdown(itinerary_md))


### Day 1: Arrival and Night Cruise
- **Activity Name**: Seine River Night Cruise
- **Location**: Port de la Bourdonnais
- **Description**: Relax and enjoy a beautiful view of Paris's landmarks illuminated at night on a comfortable boat ride along the Seine River.
- **Why Suitable**: Perfect to unwind after traveling, ideal for adults who appreciate scenic experiences.
- **Potential Reviews**: Highly rated for romantic ambiance and unique perspectives of the Eiffel Tower and Notre Dame.

### Day 2: Exploration of the 12th District
- **Activity Name**: Visit Parc de Bercy 
- **Location**: 8 Cour de Bercy
- **Description**: Enjoy a morning stroll or a picnic in the park featuring beautiful gardens, open spaces, and outdoor activities.
- **Why Suitable**: Offers a peaceful atmosphere, great for relaxation and leisurely walks.
- **Potential Reviews**: Visitors praise its tranquility and lovely gardens.

- **Restaurant Recommendation**: **Bistrot Jouvence**  
  **Address**: 172 bis, rue du Faubourg Saint-Antoine, 75012 Paris  
  **Cuisine Type**: French Bistro  
  **Description**: One of the best bistros in the city, set in a beautifully preserved former pharmacy.  
  **Rating**: Highly rated for atmosphere and traditional cuisine.

### Day 3: Culture and History
- **Activity Name**: Visit the Palais Omnisports de Paris-Bercy 
- **Location**: 8 Boulevard de Bercy
- **Description**: Check out the event schedule for concerts or shows occurring during the trip.
- **Why Suitable**: Great for those who enjoy live entertainment.
- **Potential Reviews**: Known for excellent acoustics and a variety of events.

- **Restaurant Recommendation**: **Restaurant Passerini**  
  **Address**: 65, rue Traversière, 75012 Paris  
  **Cuisine Type**: Italian  
  **Description**: Known for a cozy atmosphere with exquisite Italian cuisine, very popular among locals.  
  **Rating**: Highly regarded for authentic flavors and quality.

### Day 4: Art and Museums
- **Activity Name**: Explore the Musée des Arts Forains
- **Location**: 53 Avenue des Terroirs de France
- **Description**: An interactive museum showcasing carnival arts and fairground attractions.
- **Why Suitable**: Engaging and nostalgic, perfect for culture lovers.
- **Potential Reviews**: Visitors find it enchanting and appreciate the hands-on nature of the exhibits.

- **Restaurant Recommendation**: **Bistro S**  
  **Address**: 7, rue Saint-Nicolas, 75012 Paris  
  **Cuisine Type**: Japanese-Fusion  
  **Description**: A unique bistro offering a blend of Japanese and French cuisines.  
  **Rating**: Highly praised for creativity and taste.

### Day 5: Culinary Delights
- **Activity Name**: Culinary Class or Food Tour in the 12th District  
- **Location**: Various cooking schools or culinary spots  
- **Description**: Participate in a French cooking class or take a guided food tour sampling local treats.
- **Why Suitable**: Ideal for food lovers wanting to indulge.
- **Potential Reviews**: Praised for knowledgeable instructors and delicious outcomes.

### Day 6: Day Trip to Nearby Attractions
- **Activity Name**: Visit Château de Vincennes
- **Location**: 2 Avenue de Paris, Vincennes
- **Description**: Explore the historic castle with beautiful grounds and architectural significance.
- **Why Suitable**: A blend of history and peaceful surroundings.
- **Potential Reviews**: Noted for impressive architecture and less crowded nature.

- **Restaurant Recommendation**: **Au Trou Gascon**  
  **Address**: 40, rue Taine, 75012 Paris  
  **Cuisine Type**: French  
  **Description**: A well-known spot for hearty French cooking, famous for its traditional recipes.  
  **Rating**: Very well-reviewed for quality and ambiance.

### Day 7: Leisure and Wrap-up
- **Activity Name**: Stroll through Marché d'Aligre 
- **Location**: Place d'Aligre
- **Description**: A vibrant market perfect for last-minute exploration and cultural immersion.
- **Why Suitable**: Engaging atmosphere, ideal for picking up souvenirs and fresh produce.
- **Potential Reviews**: Loved for its local vibe and variety of offerings.

- **Restaurant Recommendation**: **Bistro Godaille**  
  **Address**: 9, rue Antoine Vollon, 75012 Paris  
  **Cuisine Type**: French Bistro  
  **Description**: A cozy bistro known for its casual vibe and delicious meals, conveniently located.  
  **Rating**: Highly praised by regulars for its welcoming atmosphere and tasty dishes.

Enjoy the trip and indulge in the culinary and cultural treasures that Paris, especially the 12th district, has to offer!
